## Perspectief 1: Waarom investeren in persoonlijke hulpdiensten het meeste geluk oplevert

Geluk hangt af van allerlei factoren, zoals inkomen, gezondheid, gezin en werk. Uit onze analyse blijkt dat één specifieke factor extra belangrijk is: *sociale hulpdiensten. Hiermee bedoelen we diensten waarbij mensen persoonlijk geholpen worden door professionals, zoals thuiszorg, psychische hulp (GGZ), coaching bij werkloosheid, kinderopvang en begeleiding in wijken. Deze diensten worden ook *“hulp in natura”** genoemd: mensen krijgen geen geld, maar concrete begeleiding en zorg.

We baseren onze analyse op data uit 77 landen tussen 2011 en 2016. Hieronder laten we duidelijk zien waarom investeren in deze diensten zoveel verschil maakt voor het geluk in een land.

## Argument 1: Sociale hulpdiensten zorgen voor meer geluk


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import plotly.graph_objects as go

# ───── 1. Data inlezen ──────────────────────────────────────
MENTAL_CSV = "Data/mental_welfare_dataset.csv"
CPDS_CSV   = "Data/cpds-1960-2022-update-2024-2.csv"

mental = pd.read_csv(MENTAL_CSV).rename(columns={"Unnamed: 0": "drop"}, errors="ignore")
cpds = pd.read_csv(CPDS_CSV)
cpds.columns = [c.lower() for c in cpds.columns]

# ───── 2. Harmonisatie ──────────────────────────────────────
rename = {"United States of America": "United States",
          "Czech Republic": "Czechia", "Russian Federation": "Russia"}
for d in (mental, cpds):
    d["country"] = d["country"].replace(rename)

mental.rename(columns={"life_ladder": "ladder"}, inplace=True)

# ───── 3. Variabelen ────────────────────────────────────────
social_vars_all = [s.lower() for s in [
    "sstran", "socexp_t_pmp", "socexp_c_pmp", "socexp_k_pmp",
    "oldage_pmp", "survivor_pmp", "incapben_pmp", "health_pmp",
    "family_pmp", "fallow_pmp", "mpleave_pmp", "childcare_pmp",
    "homehelp_pmp", "unemp_pmp", "almp_pmp", "training_pmp",
    "housing_pmp", "othsocx_pmp"
]]
social_vars = [v for v in social_vars_all if v in cpds.columns]

# ───── 4. Merge ─────────────────────────────────────────────
merged = mental.merge(cpds[["country", "year"] + social_vars],
                      on=["country", "year"], how="inner").dropna()

# ───── 5. Correlatie berekenen en filteren ──────────────────
r_values = []
filtered_vars = []
for s in social_vars:
    r = pearsonr(merged["ladder"], merged[s])[0]
    if r >= 0.4:  # Alleen positieve verbanden >= 0.4
        r_values.append(r)
        filtered_vars.append(s)

# ───── 6. Labels en uitleg ─────────────────────────────────
soc_lbl = {
    "sstran": "Soc. transfers %GDP",
    "socexp_t_pmp": "Total soc. exp.",
    "socexp_c_pmp": "Cash soc. exp.",
    "socexp_k_pmp": "In-kind soc. exp.",
    "oldage_pmp": "Old-age ben.",
    "survivor_pmp": "Survivor ben.",
    "incapben_pmp": "Incapacity ben.",
    "health_pmp": "Health exp.",
    "family_pmp": "Family ben.",
    "fallow_pmp": "Family allowance",
    "mpleave_pmp": "Mat./Pat. leave",
    "childcare_pmp": "Child-care ben.",
    "homehelp_pmp": "Home-help ben.",
    "unemp_pmp": "Unemployment ben.",
    "almp_pmp": "ALMP programmes",
    "training_pmp": "Training prog.",
    "housing_pmp": "Housing ben.",
    "othsocx_pmp": "Other soc. exp."
}

soc_expl = {
    "sstran": "Overheidsuitgaven aan sociale transfers als % van het BBP.",
    "socexp_t_pmp": "Totale sociale overheidsuitgaven per hoofd van de bevolking.",
    "socexp_c_pmp": "Sociale uitgaven in geld, per persoon.",
    "socexp_k_pmp": "Sociale uitgaven in natura (zoals gezondheidszorg), per persoon.",
    "oldage_pmp": "Overheidsuitgaven aan pensioenen, per persoon.",
    "survivor_pmp": "Uitgaven aan nabestaanden, per persoon.",
    "incapben_pmp": "Uitgaven aan arbeidsongeschiktheid, per persoon.",
    "health_pmp": "Overheidsuitgaven aan gezondheidszorg, per persoon.",
    "family_pmp": "Overheidsuitgaven aan gezinsvoordelen, per persoon.",
    "fallow_pmp": "Kinderbijslag en soortgelijke voordelen, per persoon.",
    "mpleave_pmp": "Uitgaven aan ouderschapsverlof, per persoon.",
    "childcare_pmp": "Uitgaven aan kinderopvang, per persoon.",
    "homehelp_pmp": "Uitgaven aan thuishulp voor ouderen/zieken.",
    "unemp_pmp": "Uitgaven aan werkloosheidsuitkeringen, per persoon.",
    "almp_pmp": "Actief arbeidsmarktbeleid, zoals training/subsidies.",
    "training_pmp": "Training- en omscholingsprogramma's, per persoon.",
    "housing_pmp": "Uitgaven aan huisvestingssteun, per persoon.",
    "othsocx_pmp": "Overige sociale uitgaven, per persoon."
}

col_labels = [soc_lbl.get(s, s) for s in filtered_vars]

# ───── 7. Sorting toepassen ─────────────────────────────────
combined = sorted(zip(col_labels, r_values, filtered_vars), key=lambda x: x[1])
col_labels_sorted = [x[0] for x in combined]
r_values_sorted = [x[1] for x in combined]
filtered_vars_sorted = [x[2] for x in combined]

# ───── 8. Hovertekst opnieuw genereren ──────────────────────
hover_text_sorted = []
for i in range(len(r_values_sorted)):
    uitleg = soc_expl.get(filtered_vars_sorted[i], "")
    tekst = (
        f"<b>{col_labels_sorted[i]}</b><br>"
        f"{uitleg}<br><br>"
        f"Verband: {r_values_sorted[i]:.2f}<br>"
        f"Hoger → meer geluk"
    )
    hover_text_sorted.append(tekst)

# ───── 9. Heatmap tekenen ───────────────────────────────────
fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=[r_values_sorted],
    x=col_labels_sorted,
    y=["Geluksscore"],
    colorscale='reds',
    zmin=0,
    zmax=1,
    text=[hover_text_sorted],
    hoverinfo="text",
    showscale=False  # Geen kleurbar
))

fig.update_layout(
    title="Sterkte van positieve verbanden tussen overheidsuitgaven en geluk",
    xaxis_title="Soort overheidsuitgave (oplopend verband)",
    xaxis_tickangle=-45,
    width=1100,
    height=300,
    font=dict(size=12),
    plot_bgcolor='white',
    margin=dict(l=100, r=100, t=100, b=120)
)

fig.show()


**Visualisatie 1.1**  
Deze heatmap vergelijkt elf soorten sociale uitgaven met het geluk in verschillende landen. Hoe donkerder rood, hoe sterker het verband met geluk.

*Wat zien we in deze heatmap?*
Het vakje voor sociale hulpdiensten (in-natura social expenditure) kleurt duidelijk het donkerst rood. De correlatie met geluk is uitzonderlijk hoog (Pierson-correlatie = 0,80). Andere categorieën, zoals uitkeringen of pensioenen, zijn zichtbaar lichter en hebben dus een minder sterke relatie met geluk.
Kort gezegd: van alle soorten sociale uitgaven leidt investeren in persoonlijke hulpdiensten het duidelijkst tot meer geluk.


## Argument 2: Hoe meer hulpdiensten, hoe hoger het geluk en hoe lager het suïcidecijfer



In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Dataset inladen
pad = "Data/volledige_gecombineerde_dataset_met_family.csv"
df = pd.read_csv(pad)

# Filter op 2012
df_2012 = df[df['year'] == 2012].dropna(subset=[
    'socexp_k_pmp',
    'suicides_per_100k',
    'ladder_y',
    'total_mental_illness_prev'
])

# Functie voor regressielijn en correlatie
def bereken_regressie(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    x_vals = np.linspace(x.min(), x.max(), 100)
    y_vals = slope * x_vals + intercept
    corr = round(x.corr(y), 2)
    return x_vals, y_vals, corr

# Keuzevariabelen en labels
variabelen = {
    'suicides_per_100k': 'Suïcidecijfers (per 100.000 inwoners)',
    'ladder_y': 'Geluksscore (Cantril ladder, 0–10)',
    'total_mental_illness_prev': 'Totaal mentale aandoeningen (%)'
}

# Standaard indicator
y_default = 'suicides_per_100k'

# Eerste keer plotten
x_data = df_2012['socexp_k_pmp']
y_data = df_2012[y_default]
x_vals, y_vals, corr = bereken_regressie(x_data, y_data)

fig = go.Figure()

# Scatterplot
fig.add_trace(go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(size=10, color='steelblue', line=dict(width=1, color='black')),
    text=df_2012['country'],
    hovertemplate="<b>Land:</b> %{text}<br>Sociale uitgaven in natura: %{x:.2f}% van BBP<br>Waarde: %{y:.2f}<extra></extra>",
    name='Landen'
))

# Trendlijn
fig.add_trace(go.Scatter(
    x=x_vals,
    y=y_vals,
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Trendlijn'
))

# Dropdownmenu met duidelijke labels + correlatie in titel
dropdown_buttons = []
for var, label in variabelen.items():
    y_data = df_2012[var]
    x_vals, y_vals, corr = bereken_regressie(x_data, y_data)

    dropdown_buttons.append(dict(
        label=f"{label} (r = {corr})",
        method="update",
        args=[
            {"x": [x_data, x_vals],
             "y": [y_data, y_vals]},
            {"yaxis": {"title": label},
             "title": f"{label} versus sociale uitgaven in natura (2012) — r = {corr}"}
        ]
    ))

# Layout
fig.update_layout(
    title=f"{variabelen[y_default]} versus sociale uitgaven in natura (2012) — r = {corr}",
    xaxis_title="Sociale uitgaven in natura als percentage van BBP",
    yaxis_title=variabelen[y_default],
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="up",
            x=1.2,
            xanchor="right",
            y=-0.2,
            yanchor="top",
            showactive=True
        )
    ],
    template="simple_white"
)

fig.show()


**Visualisatie 1.2**  
Met deze interactieve grafiek kun je het aandeel van het inkomen (BBP) dat naar sociale hulpdiensten gaat vergelijken met drie variabelen: geluk, zelfmoordcijfers en mentale aandoeningen.

*Wat zien we?*
- ⁠Kies je voor *geluksscore* (r = 0,73), dan zie je duidelijk: hoe meer procent BBP aan hulpdiensten, hoe hoger de geluksscore. Landen die 10% of meer investeren scoren gemiddeld 7,5 uit 10, terwijl landen onder 6% niet verder komen dan 5,5.
- Zet je de knop op *suïcide per 100.000 mensen* (r = -0,14), dan gaat de lijn iets omlaag: meer investering in hulpdiensten betekent dus minder zelfdodingen.
- ⁠Kies je *mentale aandoeningen* (r = 0,35), dan stijgt de lijn licht. Dit betekent vooral dat landen met betere hulpdiensten problemen beter herkennen en registreren.

meer investeren in persoonlijke hulpdiensten verhoogt niet alleen het gemiddelde geluk, maar zorgt ook voor minder suïcides en betere mentale zorg.

## Argument 3: Persoonlijke hulpdiensten zijn effectiever dan directe uitkeringen

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Dataset inladen
pad = "Data/volledige_gecombineerde_dataset.csv"  # Pas pad aan indien nodig
df = pd.read_csv(pad)

# Filteren op compleet jaar
df_2012 = df[df['year'] == 2012].dropna(subset=[
    'socexp_c_pmp',
    'socexp_k_pmp',
    'ladder_y'  # Dit is de juiste kolomnaam
])

# Functie voor regressielijn
def regressielijn(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    x_vals = np.linspace(x.min(), x.max(), 100)
    y_vals = slope * x_vals + intercept
    return x_vals, y_vals

# Data voorbereiden
x_cash = df_2012['socexp_c_pmp']
x_natura = df_2012['socexp_k_pmp']
y_geluk = df_2012['ladder_y']


# Regressielijnen
x_vals_cash, y_vals_cash = regressielijn(x_cash, y_geluk)
x_vals_natura, y_vals_natura = regressielijn(x_natura, y_geluk)

# Figuur
fig = go.Figure()

# Cash-uitgaven
fig.add_trace(go.Scatter(
    x=x_cash,
    y=y_geluk,
    mode='markers',
    marker=dict(color='blue'),
    text=df_2012['country'],
    name='Cash-uitgaven (% BBP)',
    hovertemplate="<b>%{text}</b><br>Cash-uitgaven: %{x:.2f}% van BBP<br>Geluksscore: %{y:.2f}<extra></extra>"
))

# In-kind-uitgaven
fig.add_trace(go.Scatter(
    x=x_natura,
    y=y_geluk,
    mode='markers',
    marker=dict(color='orange'),
    text=df_2012['country'],
    name='In-kind-uitgaven (% BBP)',
    hovertemplate="<b>%{text}</b><br>In-kind-uitgaven: %{x:.2f}% van BBP<br>Geluksscore: %{y:.2f}<extra></extra>"
))

# Trendlijn cash
fig.add_trace(go.Scatter(
    x=x_vals_cash,
    y=y_vals_cash,
    mode='lines',
    line=dict(color='blue', dash='dash'),
    name='Trend cash-uitgaven'
))

# Trendlijn in-kind
fig.add_trace(go.Scatter(
    x=x_vals_natura,
    y=y_vals_natura,
    mode='lines',
    line=dict(color='orange', dash='dash'),
    name='Trend in-kind-uitgaven'
))

# Layout
fig.update_layout(
    title="Relatie tussen sociale uitgaven en geluk (2012)",
    xaxis_title="Sociale uitgaven als percentage van BBP",
    yaxis_title="Gemiddelde geluksscore (0–10)",
    template="simple_white"
)

fig.show()


**Visualisatie 2.3**  
Deze grafiek toont een directe vergelijking tussen investeringen in persoonlijke hulpdiensten (geel) en directe uitkeringen in geld (blauw).

*Wat zien we hier?*
- ⁠De gele punten (hulpdiensten, r = 0,73) stijgen sterk: hoe meer hulpdiensten, hoe hoger de gemiddelde geluksscore.
- ⁠De blauwe punten (cash-uitkeringen, r = 0,06) blijven vrijwel vlak: meer uitkeringen leiden nauwelijks tot extra geluk.
- Frankrijk besteedt bijvoorbeeld in 2012 zo’n 13% van het BBP aan directe uitkeringen en haalt een geluksscore van 6,5. Zweden investeert minder in uitkeringen (9%), maar juist veel in hulpdiensten en scoort daarmee hoger: ongeveer 7,4.

De grafiek laat duidelijk zien: persoonlijke hulpdiensten verbeteren het geluk aanzienlijk meer dan directe financiële steun.


## Economisch rendement en beleidsimplicaties

Gelukkige mensen zijn productiever. Uit de recente grote studie van Fang, Veenhoven & Burger (2025) blijkt dat elke flinke stap omhoog in gemiddelde geluksscore leidt tot ongeveer 35% extra productiviteit. Deze studie bundelt resultaten uit 27 landen en 197 verschillende onderzoeken, en laat daarmee zien dat investeren in geluk letterlijk loont.


## Wat betekent dit voor Nederland?

- ⁠Als Nederland slechts 1% extra van het nationale inkomen (BBP) investeert in persoonlijke hulpdiensten, stijgt de gemiddelde geluksscore naar verwachting al met ongeveer 0,25 punt.
- ⁠Door de verhoogde productiviteit verdient deze investering zichzelf uiteindelijk terug. Meer geluk betekent niet alleen een prettiger leven, maar ook beter werk, minder zorgkosten en hogere belastingopbrengsten.
- ⁠Nederland kan dit eenvoudig monitoren met drie concrete indicatoren: gemiddelde geluksscore, aantal zelfdodingen, en aantal mensen met mentale problemen.

Meer investeren in persoonlijke hulpdiensten maakt Nederland dus niet alleen gelukkiger, maar ook economisch gezonder en veerkrachtiger.